## Day 5: Supply Stacks 

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:
```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```
In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:
```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 ]
```
In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:
```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
```
Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:
```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
```
Finally, one crate is moved from stack 1 to stack 2:
```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
```
The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

After the rearrangement procedure completes, what crate ends up on top of each stack?

### Approach

Need to parse the input... moderately horrible today! 
1. For the stacks, can use fixed width processing 
2. Number of stacks: find the stack identifier line (first char: "1")
3. look out for "[" vs "move" as first token

Use array of arrays (well, list of lists in python) to hold stacks - note 1 index on instructions, rather than zero index.
Reverse them - i.e. append and remove from tail. Will be slightly more efficient. Python does have a pop() function that can easily remove last element of a list

## Input data layout analysis
```
00000000001
12345678901
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

Stack 1 char @ 2
Stack 2 char @ 6
Stack 3 char @ 10

Charater at 2 + 4(n -1) 





In [2]:
testdata = """    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2"""

In [22]:
#pre-process input and split into: number of stack, the stack text, the move text

def processinput(input:str)->(int,[str],[str]):
    maxStackID = 0
    stacksString = []
    moves = []
    for l in str.splitlines(input):
        #get first non-whitespace character
        cleanstring = l.strip()


        if len(cleanstring) > 0:
            firstchar = cleanstring[0]
            match firstchar:
                case '[':
                    #stacks
                    stacksString.append(l)
                case '1':
                    #stack counters
                    stackIDs = [int(x) for x in cleanstring.split()]
                    maxStackID = max(stackIDs)
                case 'm':
                    #move instructions
                    moves.append(l)

    return (maxStackID, stacksString, moves)


#testdata
print(processinput(testdata))

#for real
input = open("day5input.txt").read()
maxStackID, stacksString, moves = processinput(input)
print(maxStackID, len(stacksString), len(moves))



(3, ['    [D]    ', '[N] [C]    ', '[Z] [M] [P]'], ['move 1 from 2 to 1', 'move 3 from 1 to 3', 'move 2 from 2 to 1', 'move 1 from 1 to 2'])
9 8 502


In [34]:
def processStacks(stacksString: [str], numStacks:int)->[[str]]:
    #split the input input lines of stacks into an array of arrays
    stacks = [[] for i in range(numStacks)] #set up the empty stacks
    for l in stacksString:
        #make it fixed width, so we don't have to do bound checks later
        width = 2 + 4*numStacks #should -1, but doesn't matter for padding
        l.ljust(width)
        for i in range(numStacks): #zero index
            charposition = 2 + 4*i - 1
            stackchar = l[charposition]
            if stackchar != ' ':
                stacks[i].append(stackchar)
    #reverse the stack
    [stack.reverse() for stack in stacks]
    return stacks


#test
maxStackID, stacksString, moves = processinput(testdata)
stacklists = processStacks(stacksString, maxStackID)
print(stacklists)


[['Z', 'N'], ['M', 'C', 'D'], ['P']]


In [35]:


def topItemsFromStacks(stacks:[[str]])->str:
    #our stacks are reverse, so last item in each stack
    output = ''
    for stack in stacks:
        output += stack[-1]
    return output

#test
print(topItemsFromStacks(stacklists)) 


NDP


In [36]:
def moveStacks(stacks:[[str]],moves:[str])->[[str]]:
    for moveStr in moves:
        #example: move 1 from 2 to 1
        moveInstruction = moveStr.split()
        numItems = int(moveInstruction[1])
        fromStack = int(moveInstruction[3]) - 1
        toStack = int(moveInstruction[5]) - 1
        #print((numItems,fromStack,toStack)) 
        #items move one at a time!
        for x in range(numItems):
            item = stacks[fromStack].pop()
            stacks[toStack].append(item)
    
    return stacks

finalstacks = moveStacks(stacklists,moves)
print(finalstacks)
print(topItemsFromStacks(finalstacks))



[['C'], ['M'], ['P', 'D', 'N', 'Z']]
CMZ


In [37]:
#put it all together
input = open("day5input.txt").read()
maxStackID, stacksString, moves = processinput(input)
stacklists = processStacks(stacksString, maxStackID)
finalstacks = moveStacks(stacklists,moves)
print(finalstacks)
print(topItemsFromStacks(finalstacks))





[['B', 'Q', 'N', 'Q', 'W'], ['V', 'F', 'Q', 'M', 'F', 'G', 'H'], ['T'], ['L'], ['C', 'N', 'P', 'J', 'B', 'P', 'G', 'H', 'M', 'B', 'M', 'H', 'M', 'T', 'S', 'H', 'P', 'M', 'T', 'B', 'M', 'F', 'G', 'W', 'N', 'P', 'R'], ['M', 'M'], ['L', 'B', 'D', 'N', 'H', 'Z'], ['G', 'D', 'L', 'R'], ['D', 'V', 'C']]
WHTLRMZRC


## Part Two 

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a CrateMover 9000 - it's a CrateMover 9001.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:
```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
Moving a single crate from stack 2 to stack 1 behaves the same as before:
```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:
```
        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3
```
Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:
```
        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3
```
Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:
```
        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3
```
In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. After the rearrangement procedure completes, what crate ends up on top of each stack?



In [43]:
def moveStacks9001(stacks:[[str]],moves:[str])->[[str]]:
    for moveStr in moves:
        #example: move 1 from 2 to 1
        moveInstruction = moveStr.split()
        numItems = int(moveInstruction[1])
        fromStack = int(moveInstruction[3]) - 1
        toStack = int(moveInstruction[5]) - 1
        #print((numItems,fromStack,toStack)) 
        #items move all at once - so need to move slices
        items = stacks[fromStack][-numItems:]
        stacks[fromStack] = stacks[fromStack][:-numItems]
        stacks[toStack].extend(items)
    
    return stacks

#test
input = testdata
maxStackID, stacksString, moves = processinput(input)
stacklists = processStacks(stacksString, maxStackID)
finalstacks = moveStacks9001(stacklists,moves)
print(finalstacks)
print(topItemsFromStacks(finalstacks))


[['M'], ['C'], ['P', 'Z', 'N', 'D']]
MCD


In [44]:
#part 2 final run
input = open("day5input.txt").read()
maxStackID, stacksString, moves = processinput(input)
stacklists = processStacks(stacksString, maxStackID)
finalstacks = moveStacks9001(stacklists,moves)
print(finalstacks)
print(topItemsFromStacks(finalstacks))

[['V', 'M', 'P', 'C', 'G'], ['N', 'N', 'B', 'H', 'S', 'D', 'M'], ['P'], ['M'], ['P', 'G', 'P', 'L', 'T', 'M', 'J', 'H', 'B', 'B', 'N', 'W', 'B', 'Q', 'R', 'B', 'Q', 'D', 'M', 'F', 'H', 'V', 'H', 'F', 'C', 'F', 'L'], ['H', 'W'], ['L', 'Q', 'T', 'R', 'Z', 'N'], ['T', 'M', 'D', 'M'], ['M', 'G', 'G']]
GMPMLWNMG
